<a href="https://colab.research.google.com/github/Kotwic4/fake-news-detector/blob/colab/colab/autokeras_overfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests numpy matplotlib pandas git+git://github.com/keras-team/autokeras@master#egg=autokeras

  Cloning git://github.com/keras-team/autokeras (to revision master) to /tmp/pip-install-7pkk56o7/autokeras
  Running command git clone -q git://github.com/keras-team/autokeras /tmp/pip-install-7pkk56o7/autokeras
     |████████████████████████████████| 86.3MB 96kB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 3.8MB 28.6MB/s 
     |████████████████████████████████| 450kB 12.4MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
  Created wheel for autokeras: filename=autokeras-1.0.0a0-cp36-none-any.whl size=50331 sha256=9261e1cbd5e354b3662e45faafedf70d9aa6411a2c4fb2d70b65fe90a3667ed0
  Stored in directory: /tmp/pip-ephem-wheel-cache-rfyiv0oe/wheels/55/b0/bc/7304f4fcafa1a7f163e4e4da7cb951f17b1cacf2e76b7d3146
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=27c8970129b5e7f7db281f8e2673baa85f0288c35e0691be1c48a538a7ef53c9
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c7

In [1]:
!nvidia-smi

Tue Dec 10 21:40:42 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!wget - O politifact_fake.csv https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/politifact_fake.csv
!wget - O politifact_real.csv https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/politifact_real.csv

--2019-12-10 21:40:48--  http://-/
Resolving - (-)... failed: Name or service not known.
wget: unable to resolve host address ‘-’
--2019-12-10 21:40:48--  http://o/
Resolving o (o)... failed: Name or service not known.
wget: unable to resolve host address ‘o’
--2019-12-10 21:40:48--  http://politifact_fake.csv/
Resolving politifact_fake.csv (politifact_fake.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘politifact_fake.csv’
--2019-12-10 21:40:48--  https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/politifact_fake.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3286418 (3.1M) [text/plain]
Saving to: ‘politifact_fake.csv.1’

politifact_fake.csv 100%[===================>]   3.13M  --.-KB/s    in 0.08s   

2019

In [3]:
!ls -l

total 22608
-rw-r--r--  1 root root 3286418 Dec 10 20:38 politifact_fake.csv
-rw-r--r--  1 root root 3286418 Dec 10 21:40 politifact_fake.csv.1
-rw-r--r--  1 root root 8278658 Dec 10 20:38 politifact_real.csv
-rw-r--r--  1 root root 8278658 Dec 10 21:40 politifact_real.csv.1
drwxr-xr-x  1 root root    4096 Dec  6 16:53 sample_data
drwxr-xr-x 15 root root    4096 Dec 10 21:39 text_classifier


In [0]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import os
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from autokeras import TextClassifier

In [0]:
DATASET_NAME = 'politifact'
DATASET_PATH = './{}'.format(DATASET_NAME)
REAL_DATA_PATH = '{}_real.csv'.format(DATASET_PATH)
FAKE_DATA_PATH = '{}_fake.csv'.format(DATASET_PATH)

In [0]:
fake_arts = pd.read_csv(FAKE_DATA_PATH, na_values=['nan'], keep_default_na=False)
real_arts = pd.read_csv(REAL_DATA_PATH, na_values=['nan'], keep_default_na=False)

In [0]:
def parse_art_data_frame(df):
    return [{'id': id, 'url': url, 'title': title} for id, url, title, tweets in df.values]
    
fake_arts_with_content = parse_art_data_frame(fake_arts)
real_arts_with_content = parse_art_data_frame(real_arts)

In [0]:
fake_data = [(art, 'fake') for art in fake_arts_with_content]
real_data = [(art, 'real') for art in real_arts_with_content]

In [0]:
all_data = fake_data + real_data
X = [x for x,y in all_data]
y = [y for x,y in all_data]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))

In [10]:
print(len(train_data))
print(len(test_data))
print(len(train_data) + len(test_data))
print(len(fake_data) + len(real_data))

844
212
1056
1056


In [11]:
print(len([y for y in y_train if y == 'fake']))
print(len([y for y in y_test if y == 'fake']))

print(len([y for y in y_train if y == 'real']))
print(len([y for y in y_test if y == 'real']))

334
98
510
114


In [0]:
def clear_text(text):
    return ' '.join(text.split())

class Classifier():
    def __init__(self, classifier):
        self.classifier = classifier
        
    def predict(self, text):
        text = clear_text(text)
        sentence = Sentence(text)
        self.classifier.predict(sentence)
        return sentence.labels[0]
    
def train_model(train_data,
               data_folder = '.',
               model_folder = '.',
               max_epochs=1
               ):
    clf = TextClassifier(verbose=True)
    train_data
    clf.fit(x=x_train, y=y_train)
    return Classifier(clf)

def calculate_metrics(y_true, y_pred, pos_label = 'fake'):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=pos_label)
    recall = recall_score(y_true, y_pred, pos_label=pos_label)
    f1 = f1_score(y_true, y_pred, pos_label=pos_label)
    return acc, precision, recall, f1

def validate_model(test_data, classifier):
    y_true = [label for x, label in test_data]
    y_pred = [classifier.predict(x).value for x, label in test_data]
    acc, precision, recall, f1 = calculate_metrics(y_true, y_pred)
    print("acc: ", acc)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1: ", f1)
    return acc, precision, recall, f1

def make_test(train_data, test_data, data_folder, model_folder, max_epochs):
    classifier = train_model(train_data, data_folder, model_folder, max_epochs)
    validate_model(test_data, classifier)

In [0]:
train_content = [(x, label) for x, label in train_data] 
test_content = [(x, label) for x, label in test_data]

In [14]:
train_title = [(x['title'], label) for x, label in train_content]
test_title = [(x['title'], label) for x, label in test_content]
print(len([x for x, label in train_title if x == '']))
print(len([x for x, label in test_title if x == '']))
train_title = [(x, label) for x, label in train_title if x != ''] 
test_title = [(x, label) for x, label in test_title if x != '']
print(len(train_title))
print(len(test_title))

0
0
844
212


In [15]:
train_url = [(x['url'], label) for x, label in train_content] 
test_url = [(x['url'], label) for x, label in test_content]
print(len([x for x, label in train_url if x == '']))
print(len([x for x, label in test_url if x == '']))
train_url = [(x, label) for x, label in train_url if x != ''] 
test_url = [(x, label) for x, label in test_url if x != '']
print(len(train_url))
print(len(test_url))

39
22
805
190


In [16]:
train_mix = [(x['url'] + ', ' + x['title'], label) for x, label in train_content] 
test_mix = [(x['url'] + ', ' + x['title'], label) for x, label in test_content]
print(len([x for x, label in train_mix if x == '']))
print(len([x for x, label in test_mix if x == '']))
train_mix = [(x, label) for x, label in train_mix if x != ''] 
test_mix = [(x, label) for x, label in test_mix if x != '']
print(len(train_mix))
print(len(test_mix))

0
0
844
212


In [30]:
x_train, y_train = zip(*(train_mix+test_mix))
x_train = np.array(x_train)
y_train = np.array(y_train)
clf = TextClassifier(max_trials=30)
clf.fit(x=x_train, y=y_train, validation_split=0.2, epochs=50)

Epoch 1/50
27/27 [==============================] - 2s 81ms/step - loss: 0.6800 - accuracy: 0.5882 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 2s 58ms/step - loss: 0.6015 - accuracy: 0.6899 - val_loss: 0.5757 - val_accuracy: 0.7441
Epoch 3/50
27/27 [==============================] - 1s 55ms/step - loss: 0.4188 - accuracy: 0.9408 - val_loss: 0.4187 - val_accuracy: 0.8768
Epoch 4/50
27/27 [==============================] - 2s 58ms/step - loss: 0.2164 - accuracy: 0.9858 - val_loss: 0.3074 - val_accuracy: 0.8863
Epoch 5/50
27/27 [==============================] - 2s 57ms/step - loss: 0.0998 - accuracy: 0.9953 - val_loss: 0.2673 - val_accuracy: 0.8815
Epoch 6/50
27/27 [==============================] - 2s 59ms/step - loss: 0.0506 - accuracy: 0.9964 - val_loss: 0.2580 - val_accuracy: 0.8863
Epoch 7/50
27/27 [==============================] - 1s 55ms/step - loss: 0.0293 - accuracy: 0.9976 - val_loss: 0.2576 - val_accuracy: 0.8910
Epoch

Epoch 1/50
27/27 [==============================] - 2s 80ms/step - loss: 0.6907 - accuracy: 0.5846 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 2s 57ms/step - loss: 0.6868 - accuracy: 0.6036 - val_loss: 0.6906 - val_accuracy: 0.5403
Epoch 3/50
27/27 [==============================] - 2s 56ms/step - loss: 0.6837 - accuracy: 0.6036 - val_loss: 0.6902 - val_accuracy: 0.5403
Epoch 4/50
27/27 [==============================] - 2s 57ms/step - loss: 0.6811 - accuracy: 0.6036 - val_loss: 0.6901 - val_accuracy: 0.5403
Epoch 5/50
27/27 [==============================] - 2s 58ms/step - loss: 0.6788 - accuracy: 0.6036 - val_loss: 0.6902 - val_accuracy: 0.5403
Epoch 6/50
27/27 [==============================] - 2s 56ms/step - loss: 0.6769 - accuracy: 0.6036 - val_loss: 0.6904 - val_accuracy: 0.5403
Epoch 7/50
27/27 [==============================] - 2s 60ms/step - loss: 0.6754 - accuracy: 0.6036 - val_loss: 0.6908 - val_accuracy: 0.5403
Epoch

Epoch 1/50
27/27 [==============================] - 2s 81ms/step - loss: 0.6856 - accuracy: 0.6213 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 2s 59ms/step - loss: 0.6529 - accuracy: 0.7621 - val_loss: 0.6526 - val_accuracy: 0.6967
Epoch 3/50
27/27 [==============================] - 2s 58ms/step - loss: 0.6025 - accuracy: 0.9089 - val_loss: 0.6095 - val_accuracy: 0.8341
Epoch 4/50
27/27 [==============================] - 2s 56ms/step - loss: 0.5448 - accuracy: 0.9491 - val_loss: 0.5616 - val_accuracy: 0.8815
Epoch 5/50
27/27 [==============================] - 1s 55ms/step - loss: 0.4755 - accuracy: 0.9704 - val_loss: 0.5149 - val_accuracy: 0.8910
Epoch 6/50
27/27 [==============================] - 1s 55ms/step - loss: 0.4110 - accuracy: 0.9834 - val_loss: 0.4712 - val_accuracy: 0.8863
Epoch 7/50
27/27 [==============================] - 1s 55ms/step - loss: 0.3583 - accuracy: 0.9846 - val_loss: 0.4340 - val_accuracy: 0.8815
Epoch

Epoch 1/50
27/27 [==============================] - 3s 104ms/step - loss: 0.4076 - accuracy: 0.7964 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 2s 68ms/step - loss: 0.0573 - accuracy: 0.9893 - val_loss: 0.6029 - val_accuracy: 0.8294
Epoch 3/50
27/27 [==============================] - 2s 69ms/step - loss: 0.0202 - accuracy: 0.9988 - val_loss: 0.5807 - val_accuracy: 0.8483
Epoch 4/50
27/27 [==============================] - 2s 70ms/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 0.5597 - val_accuracy: 0.8436
Epoch 5/50
27/27 [==============================] - 2s 69ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.5375 - val_accuracy: 0.8578
Epoch 6/50
27/27 [==============================] - 2s 68ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.5142 - val_accuracy: 0.8720
Epoch 7/50
27/27 [==============================] - 2s 69ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.4903 - val_accuracy: 0.8815
Epoc

Epoch 1/50
27/27 [==============================] - 3s 106ms/step - loss: 0.3741 - accuracy: 0.8095 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 2s 72ms/step - loss: 0.0292 - accuracy: 0.9953 - val_loss: 0.5959 - val_accuracy: 0.7820
Epoch 3/50
27/27 [==============================] - 2s 70ms/step - loss: 0.0098 - accuracy: 0.9988 - val_loss: 0.5766 - val_accuracy: 0.7915
Epoch 4/50
27/27 [==============================] - 2s 70ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.5569 - val_accuracy: 0.7962
Epoch 5/50
27/27 [==============================] - 2s 70ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.5357 - val_accuracy: 0.8341
Epoch 6/50
27/27 [==============================] - 2s 70ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.5133 - val_accuracy: 0.8389
Epoch 7/50
27/27 [==============================] - 2s 69ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.4895 - val_accuracy: 0.8626
Epoc

...

Epoch 1/50
27/27 [==============================] - 3s 113ms/step - loss: 0.5059 - accuracy: 0.7160 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 2s 85ms/step - loss: 0.0611 - accuracy: 0.9834 - val_loss: 0.5431 - val_accuracy: 0.8863
Epoch 3/50
27/27 [==============================] - 2s 85ms/step - loss: 0.0102 - accuracy: 0.9964 - val_loss: 0.6209 - val_accuracy: 0.8863
Epoch 4/50
27/27 [==============================] - 2s 88ms/step - loss: 0.0051 - accuracy: 0.9976 - val_loss: 0.5520 - val_accuracy: 0.8815
Epoch 5/50
27/27 [==============================] - 2s 87ms/step - loss: 0.0043 - accuracy: 0.9988 - val_loss: 0.5700 - val_accuracy: 0.8863
Epoch 6/50
27/27 [==============================] - 2s 88ms/step - loss: 0.0011 - accuracy: 0.9988 - val_loss: 0.5954 - val_accuracy: 0.8768
Epoch 7/50
27/27 [==============================] - 2s 87ms/step - loss: 9.6240e-05 - accuracy: 1.0000 - val_loss: 0.6052 - val_accuracy: 0.8768


Epoch 1/50
27/27 [==============================] - 2s 91ms/step - loss: 0.6285 - accuracy: 0.6142 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 2s 68ms/step - loss: 0.4069 - accuracy: 0.8935 - val_loss: 0.3912 - val_accuracy: 0.8815
Epoch 3/50
27/27 [==============================] - 2s 67ms/step - loss: 0.1883 - accuracy: 0.9811 - val_loss: 0.2913 - val_accuracy: 0.8863
Epoch 4/50
27/27 [==============================] - 2s 68ms/step - loss: 0.0869 - accuracy: 0.9905 - val_loss: 0.2639 - val_accuracy: 0.8863
Epoch 5/50
27/27 [==============================] - 2s 69ms/step - loss: 0.0477 - accuracy: 0.9976 - val_loss: 0.2577 - val_accuracy: 0.8910
Epoch 6/50
27/27 [==============================] - 2s 69ms/step - loss: 0.0294 - accuracy: 0.9988 - val_loss: 0.2583 - val_accuracy: 0.8957
Epoch 7/50
27/27 [==============================] - 2s 68ms/step - loss: 0.0202 - accuracy: 0.9988 - val_loss: 0.2607 - val_accuracy: 0.8957
Epoch

Epoch 1/50
27/27 [==============================] - 2s 85ms/step - loss: 0.6888 - accuracy: 0.5941 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 2s 65ms/step - loss: 0.6839 - accuracy: 0.6012 - val_loss: 0.6902 - val_accuracy: 0.5403
Epoch 3/50
27/27 [==============================] - 2s 64ms/step - loss: 0.6812 - accuracy: 0.6036 - val_loss: 0.6898 - val_accuracy: 0.5403
Epoch 4/50
27/27 [==============================] - 2s 65ms/step - loss: 0.6791 - accuracy: 0.6036 - val_loss: 0.6896 - val_accuracy: 0.5403
Epoch 5/50
27/27 [==============================] - 2s 67ms/step - loss: 0.6774 - accuracy: 0.6036 - val_loss: 0.6895 - val_accuracy: 0.5403
Epoch 6/50
27/27 [==============================] - 2s 62ms/step - loss: 0.6746 - accuracy: 0.6036 - val_loss: 0.6895 - val_accuracy: 0.5403
Epoch 7/50
27/27 [==============================] - 2s 64ms/step - loss: 0.6736 - accuracy: 0.6036 - val_loss: 0.6896 - val_accuracy: 0.5403
Epoch

Epoch 1/50
27/27 [==============================] - 3s 118ms/step - loss: 0.5557 - accuracy: 0.7136 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
27/27 [==============================] - 2s 63ms/step - loss: 0.2270 - accuracy: 0.9515 - val_loss: 0.6339 - val_accuracy: 0.7583
Epoch 3/50
27/27 [==============================] - 2s 63ms/step - loss: 0.1542 - accuracy: 0.9976 - val_loss: 0.6157 - val_accuracy: 0.7346
Epoch 4/50
27/27 [==============================] - 2s 63ms/step - loss: 0.1154 - accuracy: 0.9976 - val_loss: 0.5994 - val_accuracy: 0.7156
Epoch 5/50
27/27 [==============================] - 2s 65ms/step - loss: 0.0919 - accuracy: 0.9988 - val_loss: 0.5831 - val_accuracy: 0.7204
Epoch 6/50
27/27 [==============================] - 2s 66ms/step - loss: 0.0808 - accuracy: 0.9988 - val_loss: 0.5641 - val_accuracy: 0.7204
Epoch 7/50
27/27 [==============================] - 2s 65ms/step - loss: 0.0679 - accuracy: 1.0000 - val_loss: 0.5444 - val_accuracy: 0.7204
Epoc

INFO:tensorflow:Oracle triggered exit
Epoch 1/50
34/34 [==============================] - 3s 82ms/step - loss: 0.6337 - accuracy: 0.6070 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
34/34 [==============================] - 2s 64ms/step - loss: 0.4190 - accuracy: 0.8892 - val_loss: 0.3203 - val_accuracy: 0.9289
Epoch 3/50
34/34 [==============================] - 2s 64ms/step - loss: 0.2026 - accuracy: 0.9697 - val_loss: 0.1885 - val_accuracy: 0.9336
Epoch 4/50
34/34 [==============================] - 2s 67ms/step - loss: 0.1076 - accuracy: 0.9820 - val_loss: 0.1246 - val_accuracy: 0.9573
Epoch 5/50
34/34 [==============================] - 2s 64ms/step - loss: 0.0639 - accuracy: 0.9915 - val_loss: 0.0851 - val_accuracy: 0.9763
Epoch 6/50
34/34 [==============================] - 2s 65ms/step - loss: 0.0426 - accuracy: 0.9924 - val_loss: 0.0616 - val_accuracy: 0.9858
Epoch 7/50
34/34 [==============================] - 2s 66ms/step - loss: 0.0312 - accuracy: 0.9953 - val_los

In [19]:
test_mix[0]

("www.breakingnews247.net/59bc13819a723/world-s-most-popular-candy-to-be-removed-from-shelves-by-october-2017.html, World's most popular candy to be removed from shelves by October 2017!",
 'fake')

In [20]:
test_mix[0][0]

"www.breakingnews247.net/59bc13819a723/world-s-most-popular-candy-to-be-removed-from-shelves-by-october-2017.html, World's most popular candy to be removed from shelves by October 2017!"

In [31]:
clf.predict(np.array([test_mix[0][0]]))

array([['fake']], dtype='<U4')

In [0]:
y_true = [label for x, label in test_mix]
x_test = [x for x, label in test_mix]
y_true = np.array(y_true)
x_test = np.array(x_test)

In [33]:
y_pre = clf.predict(x_test)
y_pre

array([['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['real'],
       ['fake'],
       ['fake'],
       ['real'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['real'],
       ['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['real'],
       ['real'

In [34]:
acc, precision, recall, f1 = calculate_metrics(y_true, y_pre)
print("acc: ", acc)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)

acc:  0.9952830188679245
precision:  0.98989898989899
recall:  1.0
f1:  0.9949238578680203
